# Extracting Representations from Deep Neural Networks





One of the most basic and useful operations on a deep network is the extraction
of representations in its hidden layers.

The reason is that representations are in themselves a remarkable object of
investigation. 
Representations are the encoding of the data at a given layer, and their categorical
content can be decoded.
The performance of this decoding can tell us how explicitly the categorical information is contained in the representation.
In this exercise we will extract representations in hidden layers when an input is given. The way to do that in Keras is not exactly transparent, but we will show it.


<img src="../NotebooksFigures/mlp.png" alt="drawing" width="800" >



In [1]:
# imports

import os
import os.path as path
from os import listdir 
from os.path import isfile, join
import numpy as np
from matplotlib import pyplot as plt
import h5py

from keras.datasets import mnist
from keras import layers
from keras.models import Model, load_model
from keras import optimizers
from keras import losses
from keras import callbacks
from keras.utils.np_utils import to_categorical

C:\Users\aless\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load data

We load a sample of data to work with. We will sample it from the test set.

In [2]:
(i_train, l_train), (i_test, l_test) = mnist.load_data()
i_train, i_test = i_train/255.0, i_test/255.0
i_train, i_test = i_train.astype(np.float32), i_test.astype(np.float32)
i_train, i_test = np.expand_dims(i_train, 3), np.expand_dims(i_test, 3)
l_train, l_test = to_categorical(l_train), to_categorical(l_test)
i_validate = i_train[50000:, :, :, :]
i_train = i_train[0:50000, :, :, :]
l_validate = l_train[50000:, :]
l_train = l_train[0:50000, :]

In [3]:
# extract a random sample of n_sample elements from test set

n_samples = 1000
idx = np.random.permutation(i_test.shape[0])[:n_samples]
samples = i_test[idx]
labels = l_test[idx]

In [4]:
print(samples.shape)
print(labels.shape)

(1000, 28, 28, 1)
(1000, 10)


# Load pre-trained model

We load a pre-trained multi-layer perceptron, similar to the one in the figure at the beginning of the notebook.

In [5]:
model = load_model(join('models', 'mlp3.h5'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
images (InputLayer)          (None, 28, 28, 1)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
hidden1 (Dense)              (None, 100)               78500     
_________________________________________________________________
hidden2 (Dense)              (None, 100)               10100     
_________________________________________________________________
hidden3 (Dense)              (None, 100)               10100     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 99,710
Trainable params: 99,710
Non-trainable params: 0
_________________________________________________________________
None


# Training

In [6]:
minibatch_size = 10
n_epochs = 5
learn_rate = 0.1
history = {}
tb_params = {
    'write_images': True, 'histogram_freq': 5, 
    'write_grads': True, 'write_graph': False
            }
activation_fcn = 'sigmoid'

In [7]:
opt = optimizers.SGD(lr=learn_rate)
model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['accuracy'])
h = model.fit(
    i_train, l_train, validation_data=(i_validate, l_validate), epochs=n_epochs,
    batch_size=minibatch_size
    )

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 6s 117us/step - loss: 3.0850e-04 - acc: 1.0000 - val_loss: 0.1208 - val_acc: 0.9777
Epoch 2/5
50000/50000 [==============================] - 6s 120us/step - loss: 3.0198e-04 - acc: 1.0000 - val_loss: 0.1205 - val_acc: 0.9778
Epoch 3/5
50000/50000 [==============================] - 6s 113us/step - loss: 2.9503e-04 - acc: 1.0000 - val_loss: 0.1211 - val_acc: 0.9782
Epoch 4/5
50000/50000 [==============================] - 5s 109us/step - loss: 2.8675e-04 - acc: 1.0000 - val_loss: 0.1210 - val_acc: 0.9781
Epoch 5/5
50000/50000 [==============================] - 5s 106us/step - loss: 2.7936e-04 - acc: 1.0000 - val_loss: 0.1210 - val_acc: 0.9782


# Extraction

In [8]:
from keras import backend as K
input_shape = i_train.shape[1:]
print(input_shape)
nsamples = 1000
test = i_test[np.random.permutation(i_test.shape[0])[0:nsamples],:,:,: ]
print(test.shape)

(28, 28, 1)
(1000, 28, 28, 1)


In [9]:
_ = [print(l.name) for l in model.layers]

images
flatten_2
hidden1
hidden2
hidden3
output


In [10]:
from keras import backend as K

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = K.function([inp, K.learning_phase()], outputs )   # evaluation function


layer_outs = functor([test, 1.])

InvalidArgumentError: images:0 is both fed and fetched.

In [25]:
import tensorflow
print(tensorflow.__version__)

1.9.0


In [25]:
_ = [print(l.shape) for l in layer_outs]

(1000, 28, 28, 1)
(1000, 784)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 10)


array([[0.94341296, 0.68029934, 0.18934679, ..., 0.5720547 , 0.8121603 ,
        0.6126356 ],
       [0.7269921 , 0.5324204 , 0.30694357, ..., 0.7420718 , 0.7657363 ,
        0.4069131 ],
       [0.13606167, 0.62172663, 0.19126253, ..., 0.89357895, 0.25269952,
        0.742206  ],
       ...,
       [0.98225236, 0.7430345 , 0.05686154, ..., 0.99125755, 0.48475155,
        0.2932695 ],
       [0.75308293, 0.554284  , 0.23079765, ..., 0.8703133 , 0.5700442 ,
        0.9495621 ],
       [0.82203573, 0.61570245, 0.03745488, ..., 0.8826881 , 0.88168645,
        0.95447797]], dtype=float32)